# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [125]:
import pandas as pd
import numpy as np
import re

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [72]:
predict=pd.read_csv('/Users/sandrine/Desktop/Prework/806_Repo/Labs/module_2/Advanced-Pandas/your-code/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [73]:
predict.columns=predict.columns.str.strip()
predict.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [74]:
predict.head(),predict.shape

(   Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  \
 0           1        337          118                  4  4.5  4.5  9.65   
 1           2        316          104                  3  3.0  3.5  8.00   
 2           3        322          110                  3  3.5  2.5  8.67   
 3           4        314          103                  2  2.0  3.0  8.21   
 4           5        330          115                  5  4.5  3.0  9.34   
 
    Research  Chance of Admit  
 0         1             0.92  
 1         1             0.72  
 2         1             0.80  
 3         0             0.65  
 4         1             0.90  ,
 (385, 9))

Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [75]:
predict.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [76]:
predict.set_index('Serial No.',inplace=True)

In [77]:
predict.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [78]:
identifier=predict['GRE Score'].astype(str)+predict['CGPA'].astype(str)
identifier.value_counts().max
#length 385 like the number of rows==> unique identifier.

<bound method Series.max of 2987.86    1
3299.16    1
3088.45    1
3309.34    1
3228.0     1
          ..
3088.36    1
2987.46    1
3068.0     1
3348.0     1
3108.33    1
Length: 385, dtype: int64>

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [79]:
predict['Identifier']=predict['GRE Score'].astype(str)+predict['CGPA'].astype(str)
predict.set_index('Identifier',inplace=True)
predict.drop(['GRE Score','CGPA'],axis=1,inplace=True)
predict.head()

,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
Identifier,,,,,,
3379.65,118,4,4.5,4.5,1,0.92
3168.0,104,3,3.0,3.5,1,0.72
3228.67,110,3,3.5,2.5,1,0.80
3148.21,103,2,2.0,3.0,0,0.65
3309.34,115,5,4.5,3.0,1,0.90


In [81]:
predict.reset_index(inplace=True)
predict.head()

,index,Identifier,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,0,3379.65,118,4,4.5,4.5,1,0.92
1,1,3168.0,104,3,3.0,3.5,1,0.72
2,2,3228.67,110,3,3.5,2.5,1,0.80
3,3,3148.21,103,2,2.0,3.0,0,0.65
4,4,3309.34,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [91]:
df=pd.read_csv('/Users/sandrine/Desktop/Prework/806_Repo/Labs/module_2/Advanced-Pandas/your-code/Admission_Predict.csv')
df.columns=df.columns.str.strip()
df.eval('CGPA>9').sum()

110

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [92]:
df.eval('CGPA>9&SOP<3.5').sum()

5

In [93]:
df['Chance of Admit'].mean()

0.7239220779220773

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [108]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return col.map(lambda x:(x-col.mean())/np.std(col))
    
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [115]:
df['CGPA_std']=standardize(df.CGPA)
df['GRE_std']=standardize(df['GRE Score'])
df['LOR_std']=standardize(df['LOR'])

In [116]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [117]:
# Libraries
from random import choices

In [118]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [120]:
df['deciding_column']=df.lookup(df.index,decision_choice)

In [121]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,-0.409947


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [122]:
df['decision']=df.deciding_column.map(lambda x : 1 if x>0.8 else 0)

In [123]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,-0.409947,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [124]:
df.decision.sum()

87

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [135]:
#df is admissions 
df.columns=df.columns.str.replace(' ','_').map(lambda x : re.sub(r'\W','',x)).str.lower()
df.columns

Index(['serial_no', 'gre_score', 'toefl_score', 'university_rating', 'sop',
       'lor', 'cgpa', 'research', 'chance_of_admit', 'cgpa_std', 'gre_std',
       'lor_std', 'deciding_column', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [142]:
pd.cut(df[df.eval('university_rating>=4')].gre_score.map(lambda x : x+10),4)
adjusted_gre
df.gre_score.map(lambda x: x+10 if df['university_rating']>)

0      (337.5, 350.0]
4      (337.5, 350.0]
9      (325.0, 337.5]
10     (337.5, 350.0]
18     (325.0, 337.5]
            ...      
370    (337.5, 350.0]
377    (325.0, 337.5]
379    (337.5, 350.0]
382    (337.5, 350.0]
384    (337.5, 350.0]
Name: gre_score, Length: 126, dtype: category
Categories (4, interval[float64]): [(299.95, 312.5] < (312.5, 325.0] < (325.0, 337.5] < (337.5, 350.0]]